# Pregunta 2 - Tarea 2
## Diego Emilio Bustamante Henríquez

### Utilidades

- `_egcd`: El algoritmo extendido de euclides. Retorna el máximo común divisor y el inverso de `a` en módulo `b`.
- `_es_potencia`: revisa si un número es potencia de algun entero menor que él.
- `_tiene_raiz_entera`: revisa si un número tiene raíz entera dado un exponente `k`.
- `_tiene_raiz_entera_intervalo`: revisa si un número tiene raíz entera dado un exponente dentro de un intervalo.
- `_fast_primality`: test de primalidad visto en clases con algunas optimizaciones.
- `_generate_prime`: genera un primo en un intervalo dado, si no existe un primo retorna `-1`.
- `_int_size`: cantidad mínima de bytes necesaria para almacenar un entero dado. 

Nota: varias de estas funciones de utilidades están basadas en mi propia tarea del ramo Diseño y Análisis de Algoritmos.

In [75]:
from random import randint
from math import gcd, log2

def _egcd(a, b):
  if a == 0: return (b, 0, 1)
  g, y, x = _egcd(b % a, a)
  return (g, x - (b // a) * y, y)

def _es_potencia(n: int):
  if n <= 3: return False
  for k in range(2, int(log2(n)) + 1):
    if _tiene_raiz_entera(n, k): return True
  return False

def _tiene_raiz_entera(n: int, k: int):
  if n <= 3: return False
  a = 1
  while pow(a,k) < n: a <<= 1
  return _tiene_raiz_entera_intervalo(n, k, a>>1, a)

def _tiene_raiz_entera_intervalo(n: int, k: int, i: int, j: int):
  while i <= j:
    if i==j: return n == pow(i,k)
    p = (i + j)>>1
    val = pow(p,k)
    if n == val: return True
    elif val < n: i = p+1
    else: j = p-1
  return False

def _fast_primality(n, k):
  if n == 1: return False
  elif n == 2: return True
  elif n%2 == 0: return False
  else:
    neg = 0
    for _ in range(1, k+1):
      a = randint(2, n-1)
      if gcd(a, n) > 1: return False
      else:
        b = pow(a, (n-1)>>1, n)
        if b == n - 1: neg = neg + 1
        elif b != 1: return False
    if neg > 0: return not _es_potencia(n)
    else: return False

def _generate_prime(a, b):
  if a%2 == 0: a += 1
  if a == b and _fast_primality(a, 100): return a
  elif a == b: return -1
  if b%2 == 0: b -= 1
  if a == b and _fast_primality(a, 100): return a
  elif a == b: return -1
  selected = set()
  candidates = b - a + 1
  for _ in range(400000):
    elem = randint(a, b)
    if elem not in selected and _fast_primality(elem, 40): return elem
    selected.add(elem)
    if candidates == len(selected): return -1
  return -1

def _int_size(i): return int(log2(i) // 8 + 1)

## Receiver


In [183]:
class RSAReceiver:
  def __init__(self, bit_len):
    if bit_len < 8: bit_len = 8
    P = _generate_prime(1<<(bit_len//2), 1<<(bit_len//2 + 1))
    Q = _generate_prime((1<<(bit_len//2 + 1)) + 1, 1<<(bit_len//2 + 2))
    self.__N = P * Q
    phi = (P-1) * (Q-1)
    self.__d = randint(1<<(bit_len//2 - 2), P - 1)
    while(gcd(self.__d, phi) != 1): self.__d = randint(1<<(bit_len//2 - 2), P - 1)
    _, e, _ = _egcd(self.__d, phi)
    self.__e = e % phi
    self.__N_size = _int_size(self.__N)

  def get_public_key(self):
    return _int_size(self.__e).to_bytes(4, "big") + \
           self.__e.to_bytes(_int_size(self.__e), "big") + \
           self.__N_size.to_bytes(4, "big") + \
           self.__N.to_bytes(_int_size(self.__N), "big")

  def decrypt(self, ciphertext):
    text = bytearray()
    for index in range(0, len(ciphertext), self.__N_size):
      int_text = pow(int.from_bytes(ciphertext[index:index+self.__N_size], "big"), self.__d, self.__N)
      text += int_text.to_bytes(_int_size(int_text), "big")
    return text.decode("utf-8")

## Sender

In [181]:
class RSASender:
  def __init__(self, public_key):
    e_size = int.from_bytes(public_key[:4], "big")
    self.__e = int.from_bytes(public_key[4:4+e_size], "big")
    self.__N_size = int.from_bytes(public_key[4+e_size:8+e_size], "big")
    self.__N = int.from_bytes(public_key[8+e_size:], "big")

  def encrypt(self, message):
    msg_bytes = bytearray(message, "utf-8")
    ciphertext = bytearray()
    for index in range(0, len(msg_bytes), self.__N_size - 1):
      ciphertext += pow(int.from_bytes(msg_bytes[index:index+self.__N_size-1], "big"), self.__e, self.__N).to_bytes(self.__N_size, "big")
    return ciphertext

In [ ]:
"""
re = RSAReceiver(1400)
key = re.get_public_key()
se = RSASender(key)
msg = se.encrypt("hola como esta mi gente wiiiiiiiiiiiii")
re.decrypt(msg)
"""